In [8]:
import pandas as pd
!pip install --upgrade scipy
!pip install --upgrade sklearn

Requirement already up-to-date: scipy in c:\users\mirela\anaconda3\lib\site-packages (1.8.0)
Processing c:\users\mirela\appdata\local\pip\cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074\sklearn-0.0-py2.py3-none-any.whl


In [9]:
df = pd.read_csv('C:\\Users\\Mirela\\PycharmProjects\\Brasilian houses to rent\\Houses-to-rent-master\\houses_to_rent_v2.csv',
                             sep=r'\s*,\s*',
                            engine='python',
                            na_values="?")
df

,city,area,rooms,bathroom,parking spaces,floor,animal,furniture,hoa (R$),rent amount (R$),property tax (R$),fire insurance (R$),total (R$)
0,SÃ£o Paulo,70,2,1,1,7,acept,furnished,2065,3300,211,42,5618
1,SÃ£o Paulo,320,4,4,0,20,acept,not furnished,1200,4960,1750,63,7973
2,Porto Alegre,80,1,1,1,6,acept,not furnished,1000,2800,0,41,3841
3,Porto Alegre,51,2,1,0,2,acept,not furnished,270,1112,22,17,1421
4,SÃ£o Paulo,25,1,1,0,1,not acept,not furnished,0,800,25,11,836
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,Porto Alegre,63,2,1,1,5,not acept,furnished,402,1478,24,22,1926
10688,SÃ£o Paulo,285,4,4,4,17,acept,not furnished,3100,15000,973,191,19260
10689,Rio de Janeiro,70,3,3,0,8,not acept,furnished,980,6000,332,78,7390
10690,Rio de Janeiro,120,2,2,2,8,acept,furnished,1585,12000,279,155,14020


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10692 entries, 0 to 10691
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   city                 10692 non-null  object
 1   area                 10692 non-null  int64 
 2   rooms                10692 non-null  int64 
 3   bathroom             10692 non-null  int64 
 4   parking spaces       10692 non-null  int64 
 5   floor                10692 non-null  object
 6   animal               10692 non-null  object
 7   furniture            10692 non-null  object
 8   hoa (R$)             10692 non-null  int64 
 9   rent amount (R$)     10692 non-null  int64 
 10  property tax (R$)    10692 non-null  int64 
 11  fire insurance (R$)  10692 non-null  int64 
 12  total (R$)           10692 non-null  int64 
dtypes: int64(9), object(4)
memory usage: 1.1+ MB


In [ ]:
target = df['rent amount (R$)'].values
target

In [ ]:
df=df.drop('rent amount (R$)', axis=1)
df

In [18]:
from sklearn.preprocessing import OneHotEncoder

oh = OneHotEncoder()
data = oh.fit_transform(df)
data

<10692x9482 sparse matrix of type '<class 'numpy.float64'>'
	with 128304 stored elements in Compressed Sparse Row format>

In [19]:
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn import tree
from sklearn import svm

In [20]:
rf = RandomForestRegressor()
cross_val_score(rf, data, target).mean()

0.7661210821303257

In [21]:
rf = RandomForestRegressor(random_state = 42)
cross_val_score(rf, data, target).mean()

0.7678127964795525

In [22]:
from sklearn.preprocessing import MaxAbsScaler
mm = MaxAbsScaler()
data1 = mm.fit_transform(data)
data1

<10692x9482 sparse matrix of type '<class 'numpy.float64'>'
	with 128304 stored elements in Compressed Sparse Row format>

In [23]:
lreg = LinearRegression()
cross_val_score(lreg, data1, target).mean()

0.5276282205934959

In [24]:
dt = tree.DecisionTreeRegressor()
cross_val_score(dt, data1, target).mean()

0.7053745046141534

In [25]:
dt.fit(data1, target)
dt.feature_importances_

array([0.00022414, 0.00085698, 0.00059025, ..., 0.        , 0.        ,
       0.        ])

In [26]:
dt.feature_importances_ > 0.0005

array([False,  True,  True, ..., False, False, False])

In [27]:
data1[:, dt.feature_importances_ > 0.0005]

<10692x203 sparse matrix of type '<class 'numpy.float64'>'
	with 67997 stored elements in Compressed Sparse Row format>

In [28]:
data1[:, dt.feature_importances_ > 0.0005].shape

(10692, 203)

In [29]:
data3 = data1[:, dt.feature_importances_ > 0.0005]

In [30]:
cross_val_score(rf, data3, target).mean()

0.8071792645294854

In [31]:
cross_val_score(dt, data3, target).mean()

0.7637706413151842

In [32]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

train_test_split(range(10))

[[2, 4, 1, 8, 6, 7, 0], [3, 5, 9]]

In [33]:
trd, tsd, trt, tst = train_test_split(data3, target)

In [34]:
rf.fit(trd, trt)

RandomForestRegressor(random_state=42)

In [35]:
print(rf.predict(tsd))

[ 1635.43333333  5882.87555556  5942.75844156 ...  3180.44190476
  3000.61272646 15000.        ]


In [36]:
print(rf.score(tsd, tst))

0.7982411318246787


I have successfully use cross-validation to evaluate some estimators performance.

I also explored feature importances, and analysed the behaviour of new dataset we ve obtained.

New dataset perform better when i decided again to explore model selection.

Random Forest Regressor model performe the best on  new dataset.